<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_Cast_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---


![CC-BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png) ||
[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

#Install Swiss Ephemeris <br>

https://pypi.org/project/pyswisseph/  - Python Package Documentation <br>
https://www.astro.com/swisseph/swephinfo_e.htm <br>
https://www.astro.com/swisseph/swephprg.htm - Programmer Documentation <br>
https://pythonhosted.org/pyswisseph/swisseph-module.html

In [1]:
!pip install pyswisseph

     |████████████████████████████████| 521 kB 29.9 MB/s 
  Created wheel for pyswisseph: filename=pyswisseph-2.8.0.post1-cp37-cp37m-linux_x86_64.whl size=724385 sha256=ccb1009cb2cb777c41af262d092a4b52e7f41f4656faa7492c2266c5bba42f53
  Stored in directory: /root/.cache/pip/wheels/32/81/77/a3f7c72bd124fbdfec5f468ce0669ac09ea98a9af2b99edd61
Successfully built pyswisseph


In [2]:
!wget http://www.astro.com/ftp/swisseph/ephe/seas_18.se1
!wget http://www.astro.com/ftp/swisseph/ephe/semo_18.se1
!wget http://www.astro.com/ftp/swisseph/ephe/sepl_18.se1

--2022-01-08 03:53:22--  http://www.astro.com/ftp/swisseph/ephe/seas_18.se1
Resolving www.astro.com (www.astro.com)... 192.53.104.242
Connecting to www.astro.com (www.astro.com)|192.53.104.242|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223002 (218K)
Saving to: ‘seas_18.se1’

seas_18.se1         100%[===================>] 217.78K   593KB/s    in 0.4s    

2022-01-08 03:53:22 (593 KB/s) - ‘seas_18.se1’ saved [223002/223002]

--2022-01-08 03:53:22--  http://www.astro.com/ftp/swisseph/ephe/semo_18.se1
Resolving www.astro.com (www.astro.com)... 192.53.104.242
Connecting to www.astro.com (www.astro.com)|192.53.104.242|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1304771 (1.2M)
Saving to: ‘semo_18.se1’

semo_18.se1         100%[===================>]   1.24M  2.07MB/s    in 0.6s    

2022-01-08 03:53:23 (2.07 MB/s) - ‘semo_18.se1’ saved [1304771/1304771]

--2022-01-08 03:53:23--  http://www.astro.com/ftp/swisseph/ephe/sepl_18.se1
Resolv

In [3]:
!mkdir ephe
!mv *.se1 ephe

In [4]:
import swisseph as swe
swe.version

'2.08'

In [5]:
#Constants and Flags necessary for this application are set here
#
swe.set_ephe_path('/content/ephe') # set path to ephemeris files
# Calendar : Julian or Gregorian
SE_GREG_CAL = 1
gregflag = SE_GREG_CAL
#
# Ayanamsha type : Lahiri = 1
swe.set_sid_mode(1)  # Lahiri Aynamsha
#
# whether speed will be calculated along with position of planet
SEFLG_SPEED = int(256)
iflag = SEFLG_SPEED
#hsys = P, Placidus
#ascii P = 080
hsysP =  bytes('P', 'utf-8')

In [7]:
# Subtracts angles (eg Ayanamsha) from Longitudes 
# Required for conversion from Tropical (Western) to Sidereal (Indian) zodiac
def p21Sub (x,y):
    retVal = round(x - y,3)
    if retVal < 0:
        return retVal + 360
    else:
        return retVal

#Personal Data


## Primary Data Input

In [8]:
pName = 'itAnch'                   # person identifier
pBirthLocation = 'Anchiano, Tuscany, Italy'   # Place of Birth
LBDY = 1452                         # Year of Birth, Local Time
LBDM = 4                           # Month of Birth, Local Time
LBDD = 15                           # Day of Birth, Local Time

LBTh = 3                           # Hour of Birth, Local Time, 24 Hour Clock
LBTm = 0                           # Minute of Birth, Local Time
LBTs = 0                            # Second of Birth, Local Time

tZone = 2                         # Time Zone Offset. East of Greenwich is +ve

pBirthDate = str(LBDY)+'-'+str(LBDM)+'-'+str(LBDD)
pBirthTime = str(LBTh)+':'+str(LBTm)

print(pName,pBirthDate,pBirthTime, pBirthLocation,tZone)

itAnch 1452-4-15 3:0 Anchiano, Tuscany, Italy 2


## Data Conversions

In [9]:
# Date & Time of Birth
#Time Zone Conversion
UTCdt = swe.utc_time_zone(LBDY,LBDM,LBDD,LBTh,LBTm,LBTs,tZone)
print(UTCdt)

(1452, 4, 15, 1, 0, 0.0)


In [10]:
# Date & Time of Birth
# Convert UTC date to Julian Date
JD = swe.utc_to_jd(UTCdt[0],UTCdt[1],UTCdt[2],UTCdt[3],UTCdt[4],0,gregflag) # Second Value = 0
print (JD[0], JD[1])
# JD[0] is Ephemeris Time
# JD[1] is Universal Time 
natalUT = JD[1]
print(natalUT, 'Julian Date of DTofB in Universal Time')

2251496.5461319378 2251496.5416666665
2251496.5416666665 Julian Date of DTofB in Universal Time


In [11]:
#
# Get Lat and Long of Place of Birth
# https://pypi.org/project/geopy/
#

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Parashar21")
#location = geolocator.geocode("175 5th Avenue NYC")
#location = geolocator.geocode("36B Panditiya Road, Kolkata 700029, India")
location = geolocator.geocode(pBirthLocation)
print(location.address)

print(location.latitude, location.longitude)

#print(location.raw)
natalLAT = round(location.latitude,2)
natalLON = round(location.longitude,2)

print (natalLAT, natalLON)


Anchiano, Sant'Amato, Vinci, Firenze, Toscana, 51035, Italia
43.7997979 10.9386473
43.8 10.94


#Calculations

##Ayanamsha

In [12]:
#Ayanamsha
ayanamsha = swe.get_ayanamsa(natalUT)
print('Lahiri Ayanamsha :', ayanamsha)

Lahiri Ayanamsha : 16.215465281723574


##Position of Planets

In [13]:
#Position of Planets
#
# body 0 = Sun, 1 = Moon, ... 11 = True Node (Rahu)
bLon = []
bRet = []
for body in [0,1,2,3,4,5,6,11]:
    pData = swe.calc_ut(natalUT,body,iflag)
    bLon.append(pData[0][0])
    if pData[0][3] >= 0:
        bRet.append('False')
    else :
        bRet.append('True')
for ix in range(len(bLon)) : print(bLon[ix], bRet[ix])

25.060024639386214 False
225.64726348104176 False
7.9003638047835985 True
43.5926222952037 False
309.5210356367755 False
329.86236169953014 False
194.79588902632 True
277.2885670966194 True


##House Position and Ascendants

In [14]:
#House Position and Ascendants
P = swe.houses(natalUT,natalLAT,natalLON,hsysP)
for ix in range(len(P[0])) : print(P[0][ix])
for ix in range(len(P[1])) : print(P[1][ix])

295.47532621478933
343.78652259034817
23.998887802118055
51.62141013754302
72.86746585235147
92.53645868547517
115.47532621478933
163.7865225903481
203.99888780211805
231.62141013754302
252.86746585235147
272.53645868547517
295.47532621478933
231.62141013754302
229.18343705523412
149.50708418599856
316.7145816764518
328.73511175974954
293.0394528287018
148.73511175974954


The output generated here matches with the output generated by the Windows program swetest

#Creating py21 file

In [15]:
L1 = ''
L1 = L1 + 'La:'+str(p21Sub(P[1][0],ayanamsha))+','
L1 = L1 + 'Su:'+str(p21Sub(bLon[0],ayanamsha))+','
L1 = L1 + 'Mo:'+str(p21Sub(bLon[1],ayanamsha))+','
L1 = L1 + 'Ma:'+str(p21Sub(bLon[4],ayanamsha))+','
L1 = L1 + 'Me:'+str(p21Sub(bLon[2],ayanamsha))+','
L1 = L1 + 'Ju:'+str(p21Sub(bLon[5],ayanamsha))+','
L1 = L1 + 'Ve:'+str(p21Sub(bLon[3],ayanamsha))+','
L1 = L1 + 'Sa:'+str(p21Sub(bLon[6],ayanamsha))+','
L1 = L1 + 'Ra:'+str(p21Sub(bLon[7],ayanamsha))+','
L1 = L1 + 'Ke:'+str(p21Sub(p21Sub(bLon[7],ayanamsha),180))
L1 = L1 + '\n'
print(L1)

La:279.26,Su:8.845,Mo:209.432,Ma:293.306,Me:351.685,Ju:313.647,Ve:27.377,Sa:178.58,Ra:261.073,Ke:81.073



In [16]:
L2 = ''
L2 = L2 + 'La:'+'False'+','
L2 = L2 + 'Su:'+bRet[0]+','
L2 = L2 + 'Mo:'+bRet[1]+','
L2 = L2 + 'Ma:'+bRet[4]+','
L2 = L2 + 'Me:'+bRet[2]+','
L2 = L2 + 'Ju:'+bRet[5]+','
L2 = L2 + 'Ve:'+bRet[3]+','
L2 = L2 + 'Sa:'+bRet[6]+','
L2 = L2 + 'Ra:'+'False'+','
L2 = L2 + 'Ke:'+'False'
L2 = L2 + '\n'
print(L2)

La:False,Su:False,Mo:False,Ma:False,Me:True,Ju:False,Ve:False,Sa:True,Ra:False,Ke:False



In [17]:
L3 = pName+';'+pBirthDate+';'+pBirthTime+';'+pBirthLocation
print(L3)

itAnch;1452-4-15;3:0;Anchiano, Tuscany, Italy


In [18]:
writefile = pName+'.py21'
f1 = open(writefile, 'w')
f1.write(L1)  # python will convert \n to os.linesep
f1.write(L2)
f1.write(L3)
f1.close()

In [19]:
!cat itAnch.py21

La:279.26,Su:8.845,Mo:209.432,Ma:293.306,Me:351.685,Ju:313.647,Ve:27.377,Sa:178.58,Ra:261.073,Ke:81.073
La:False,Su:False,Mo:False,Ma:False,Me:True,Ju:False,Ve:False,Sa:True,Ra:False,Ke:False
itAnch;1452-4-15;3:0;Anchiano, Tuscany, Italy

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
!ls /content/drive/"My Drive"/Prithwis/Parashar21/py21-files

DaVinci.py21  itAnch.py21  niCha01.py21  wrCal02.py21
imCal02.py21  NDModi.py21  pmCal02.py21


In [23]:
#!cp *.py21 /content/drive/"My Drive"/Personal/Parashar21/datafiles
!cp itAnch.py21 /content/drive/"My Drive"/Prithwis/Parashar21/py21-files

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)